In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
from tqdm import tqdm
from PIL import Image
import time 
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.datasets as datasets
from torchmetrics import Accuracy, Precision, Recall, F1Score

In [3]:
path = "C:/Users/moham/Desktop/studying/Nouveau dossier/Data for Identification of Plant Leaf Diseases Using a 9-layer Deep Convolutional Neural Network/Plant_leave_diseases_dataset_with_augmentation"

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = sorted(os.listdir(data_dir))

        self.images = []
        self.labels = []

        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(data_dir, class_name)
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                self.images.append(image_path)
                self.labels.append(i)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label



In [5]:
def create_data_loaders(dataset_path, train_size=0.8, batch_size=32, shuffle=True):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    dataset = CustomDataset(dataset_path, transform=transform)

    # Define sizes for train, validation, and test sets
    train_size = int(train_size * len(dataset))
    val_size = (len(dataset) - train_size) // 2
    test_size = len(dataset) - train_size - val_size

    # Split dataset into train, validation, and test sets
    train_data, val_data, test_data = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

    # Create DataLoader instances for training, validation, and testing
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


In [6]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

def load_dataset(dataset_path, target_size=(256, 256), batch_size=32, shuffle=True): 
    
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])
    
    dataset = ImageFolder(root=dataset_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return dataloader

In [7]:
data=load_dataset(dataset_path=path,target_size=(256,256),batch_size=32,shuffle=True)

In [8]:
import matplotlib.pyplot as plt
import numpy as np

def plot_images_from_dataset(dataset, num_images=5):

    # Get a batch of images from the dataset
    images, labels = next(iter(dataset))

    # Convert PyTorch tensors to numpy arrays
    images = images.numpy()
    labels = labels.numpy()

    # Plot the images
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    for i in range(num_images):
        axes[i].imshow(np.transpose(images[i], (1, 2, 0)))  # Transpose to (height, width, channels)
        axes[i].set_title(f"Label: {labels[i]}")
        axes[i].axis('off')
    plt.show()


In [9]:

# Plot 5 images from the dataset
plot_images_from_dataset(data, num_images=5)

: 